## Base de dados

In [0]:
CREATE OR REPLACE TABLE data_lake_turing_.prata.base_cluster_user (
SELECT
a.ID,
a.Distrito,
a.Vulnerabilidade,
a.Empregado,
a.Renda_mensal,
a.Velocidade_internet,
a.Dispositivo_principal,
a.Nivel_digital,
a.Autonomia_digital,
a.Conhecimento_tecnico,
a.Interesse_curso,
a.Pessoas_na_residencia
FROM data_lake_turing_.prata.dados_usuarios_forms a)

## Pré Processamento e Ajuste das Variáveis

In [0]:
%python
import pandas as pd

table_name = 'data_lake_turing_.prata.base_cluster_user'
# ONE HOT ENCODING
df_raw = spark.read.table(table_name).toPandas()
df_encoded = pd.get_dummies(
    df_raw,
    columns=['Distrito', 'Vulnerabilidade', 'Empregado','Velocidade_internet','Dispositivo_principal','Nivel_digital','Autonomia_digital','Conhecimento_tecnico','Interesse_curso'],
    prefix=['is_', 'is_', 'is_','is_', 'is_', 'is_','is_', 'is_', 'is_']
)

df_encoded

## Cálculo de k

In [0]:
%python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Seleciona amostra
df_sample = df_encoded


# Remove duplicatas na lista de features
features = list(dict.fromkeys([
    "is__Alto de Pinheiros","is__Anhanguera","is__Aricanduva","is__Artur Alvim","is__Barra Funda","is__Bela Vista","is__Belém","is__Bom Retiro","is__Brasilândia","is__Brás","is__Butantã","is__Cachoeirinha","is__Cambuci","is__Campo Belo","is__Campo Grande","is__Campo Limpo","is__Cangaíba","is__Capão Redondo","is__Carrão","is__Casa Verde","is__Cidade Ademar","is__Cidade Dutra","is__Cidade Líder","is__Cidade Tiradentes","is__Consolação","is__Cursino","is__Ermelino Matarazzo","is__Freguesia do Ó","is__Grajaú","is__Guaianases","is__Iguatemi","is__Ipiranga","is__Itaim Bibi","is__Itaim Paulista","is__Itaquera","is__Jabaquara","is__Jaguara","is__Vila Medeiros","is__Vila Prudente","is__Vila Sônia","is__Água Rasa","is__baixa","is__media","is__não","is__sim","is__alta","is__baixa","is__media","is__celular","is__notebook","is__avançado","is__básico","is__intermediário","is__0","is__1","is__2","is__3","is__4","is__5","is__6","is__7","is__8","is__9","is__10","is__0","is__1","is__2","is__3","is__4","is__5","is__6","is__7","is__8","is__9","is__10","is__não","is__sim"
]))

X = df_sample[features]

# Normalização
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Testa vários k
ks = range(2, 11)
inertias = []
sil_scores = []

for k in ks:
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=50,max_iter=500,algorithm='elkan', random_state=123)
    labels = kmeans.fit_predict(X_scaled)
    inertias.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X_scaled, labels))

# Plot Elbow e Silhouette
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(ks, inertias, '-o')
plt.title('Elbow (Inertia)')
plt.xlabel('k')
plt.ylabel('Inertia (WCSS)')

plt.subplot(1,2,2)
plt.plot(ks, sil_scores, '-o', color='orange')
plt.title('Silhouette Score')
plt.xlabel('k')
plt.ylabel('Score')
plt.tight_layout()
plt.show()

## K-means

In [0]:
%python
k = 10
kmeans = KMeans(n_clusters=k, init='k-means++', n_init=50,max_iter=500,algorithm='elkan', random_state=123)

# Ajusta e obtém os rótulos
labels = kmeans.fit_predict(X_scaled)

df_clustered = df_sample.copy()
df_clustered['cluster'] = labels

df_clustered_v1 = df_clustered[['ID', 'cluster']]

df_clustered_v1

## Visualização dos resultados

In [0]:
%python
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis', alpha=0.7)
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.title("Distribuição dos Clusters (PCA 2D)")
plt.colorbar(label='Cluster')
plt.show()

## Criação de tabela com saída do modelo

In [0]:
%python

spark_df = spark.createDataFrame(df_clustered_v1)

# Gravar na área de tabelas gerenciadas do Databricks
spark_df.write.mode("overwrite").saveAsTable("data_lake_turing_.ouro.cluster")

## Escolha do Modelo

Escolhi a clusterização como modelo supervisionado por se enquadrar dentro da proposta do meu projeto.

Dentro da clusterização, optei pelo KNN
